In [2]:
import numpy as np
import pandas as pd
import yfinance as yf
from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential
from keras.layers import Dense, Dropout, LSTM
from pandas_datareader import data as pdr

# Data Collection and Preprocessing
yf.pdr_override()

user_input = 'TSLA'  # Replace with your desired stock ticker
start_date = '2021-01-01'
end_date = '2021-12-31'

dataframe = pdr.get_data_yahoo(user_input, start_date, end_date)
dataframe.reset_index(inplace=True)
data = dataframe.drop(dataframe.columns[[0, 2, 3, 6]], axis=1)

def training_data(df):
    ''' This function was used to create the model and train it on training data. The model has already been built and saved to avoid going through the same steps repeatedly. '''
    data_training = pd.DataFrame(df['Close'][0 : int(len(df)*0.70)])
    scaler = MinMaxScaler(feature_range=(0,1))
    data_training_array = scaler.fit_transform(data_training)

    x_train = []
    y_train = []

    for i in range(100, data_training_array.shape[0]):
        x_train.append(data_training_array[i-100 : i])
        y_train.append(data_training_array[i , 0])

    x_train, y_train = np.array(x_train), np.array(y_train)

    model = Sequential()
    model.add(LSTM(units = 50, activation = "relu", return_sequences = True,
                input_shape = (x_train.shape[1], 1)))
    model.add(Dropout(0.2))

    model.add(LSTM(units = 60, activation = "relu", return_sequences = True))
    model.add(Dropout(0.3))

    model.add(LSTM(units = 80, activation = "relu", return_sequences = True))
    model.add(Dropout(0.4))

    model.add(LSTM(units = 120, activation = "relu"))
    model.add(Dropout(0.5))

    model.add(Dense(units = 1))

    model.compile(optimizer = 'adam', loss = 'mean_squared_error')
    model.fit(x_train, y_train, epochs = 50)

    # Save the trained model
    model.save('keras_model.h5')

# Call the training function
training_data(dataframe)


[*********************100%%**********************]  1 of 1 completed


Epoch 1/50
3/3 [==============================] - 10s 227ms/step - loss: 0.1332
Epoch 2/50
3/3 [==============================] - 1s 215ms/step - loss: 0.1179
Epoch 3/50
3/3 [==============================] - 1s 216ms/step - loss: 0.0984
Epoch 4/50
3/3 [==============================] - 1s 220ms/step - loss: 0.0615
Epoch 5/50
3/3 [==============================] - 1s 212ms/step - loss: 0.0267
Epoch 6/50
3/3 [==============================] - 1s 221ms/step - loss: 0.0288
Epoch 7/50
3/3 [==============================] - 1s 213ms/step - loss: 0.0257
Epoch 8/50
3/3 [==============================] - 1s 217ms/step - loss: 0.0299
Epoch 9/50
3/3 [==============================] - 1s 216ms/step - loss: 0.0280
Epoch 10/50
3/3 [==============================] - 1s 214ms/step - loss: 0.0224
Epoch 11/50
3/3 [==============================] - 1s 222ms/step - loss: 0.0193
Epoch 12/50
3/3 [==============================] - 1s 218ms/step - loss: 0.0243
Epoch 13/50
3/3 [==============================]

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
